In [1]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
numpy.set_printoptions(threshold=sys.maxsize)

%run commons.py

4.1.1


# load Naive Bayes

In [69]:
filename = 'naive_bayes_model.sav'
gnb = pickle.load(open(filename, 'rb'))

filename_blurred = 'blurred_naive_bayes_model.sav'
gnb_blurred = pickle.load(open(filename_blurred, 'rb'))

# Sector detection

In [3]:
# parameters
percentile = 80 # percentil, with rows we accept as pallets (with more black pixel than in other rows)
frame = 10 # we decice that row is important if it neighbours in frame are also in percentil
threshold = 5 # threshold of neihtbours in frame in detecting important rows
border_size = 10 # we detecting groups of important rows (mayby pallet), 
                        #this parameter is maximal space between rows

def get_pallet_sectors(img):
    hist = [255 - np.mean(row) for row in img]
    perc = np.percentile(hist, percentile)
    hist_perc = [max(x - perc, 0) for x in hist]

    def check_frame(row_id, image):
        return np.count_nonzero(image[row_id: row_id + frame]) > threshold

    detection = [ row_id for row_id, value in enumerate(hist_perc) if check_frame(row_id, hist_perc) ]

    def check_row(list_id, rows):
        if list_id == len(rows) -1 :
            return rows[list_id] - rows[list_id - 1] < border_size
        if list_id == 0:
            return rows[list_id + 1] - rows[list_id] < border_size
        return rows[list_id] - rows[list_id - 1] > border_size or rows[list_id + 1] - rows[list_id] > border_size
        
    borders = [x for x_id, x in enumerate(detection) if check_row(x_id, detection)]
    up = borders[-2] - 4
    bottom = borders[-1] + 4
    return up, bottom

## Define pipeline functions

In [4]:
def show(img):
 cv2.imshow('image',img)
 cv2.waitKey(0)
 cv2.destroyAllWindows()

In [5]:
def detect(img, model):
    w,h,ch = img.shape
    f = features(img, channels, False)
    pred = model.predict(f)
    img_pred = np.reshape(pred, (w,h,1))
    return np.logical_not(img_pred)

In [6]:
def median_filter(img):
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(img,-1,kernel)
    return cv2.threshold(img,0.5,1.0,cv2.THRESH_BINARY)[1]

In [57]:
def rotate(img, angle):
    return  1-imutils.rotate(1-img, angle)

In [8]:
def odd(num):
    return (num % 3) == 0

In [9]:
def find_pallet_by_height(img, mask_height):
    mask_height = mask_height
    mask_width = int(mask_height * 5.556)
    mask_size = mask_height * mask_width
    hole_height = int(mask_height * 0.694)
    hole_width = int(mask_height * 1.58)
    hole_size = hole_height * hole_width
    hole_1_x = int(mask_height * 0.694)
    hole_1_y = int(mask_height * 0.306)
    hole_2_x = int(mask_height * 3.281)
    hole_2_y = hole_1_y
    best = (None, 0, (0,0))
    inverse_img = np.logical_not(img)
    img_height, img_width = inverse_img.shape
    for index, x in np.ndenumerate(inverse_img):
        x,y = index
        if odd(x) and odd(y) and y+mask_height < img_height and x+mask_width < img_width:
            frame_mask = inverse_img[y:y+mask_height, x:x+mask_width]
            hole_1_y_ = y+hole_1_y
            hole_2_y_ = y+hole_2_y
            hole_1_x_ = x+hole_1_x
            hole_2_x_ = x+hole_2_x
            hole_1_mask = inverse_img[hole_1_y_:hole_1_y_+hole_height, hole_1_x_:hole_1_x_+hole_width]
            hole_2_mask = inverse_img[hole_2_y_:hole_2_y_+hole_height, hole_2_x_:hole_2_x_+hole_width]
            frame_mask_perc = np.sum(frame_mask) / mask_size
            hole_1_mask_perc = np.sum(hole_1_mask) / hole_size
            hole_2_mask_perc = np.sum(hole_2_mask) / hole_size
            perc = frame_mask_perc - hole_1_mask_perc - hole_2_mask_perc
            if perc >= best[1]:
                best = (frame_mask, perc, (x,y))
    return best

def find_pallet(img, min_height, max_height, step, min_perc=0.0):
    best = (None, min_perc, (0,0,0))
    for mask_height in range(min_height, max_height, step):
        (best_for_height, perc, (x,y)) = find_pallet_by_height(img, mask_height)
        if perc >= best[1]:
            print("height: ", mask_height, "percent: ",perc)
            best = (best_for_height, perc, (x,y,mask_height))
    
    return (np.logical_not(best[0]), best[2])

def draw_pallet(img_full,x,y,mask_height):
    print(x,y)
    mask_width = int(mask_height * 5.556)
    color = (0,255,0)
    img_full[y:y+mask_height, x:x+1]=color
    img_full[y:y+mask_height, x+mask_width-1:x+mask_width]=color
    img_full[y:y+1, x:x+mask_width]=color
    img_full[y+mask_height-1:y+mask_height, x:x+mask_width]=color
    return img_full

def calculate_center(x,y,mask_height):
    mask_width = int(mask_height * 5.556)
    x = x + mask_width/2
    y = y + mask_height/2
    return (x,y)

In [10]:
def save(img, file, binary=True):
    res = cv2.imwrite(file, img * 255 if binary else img)
    print("saved" if res else "save error", file)

## Run pipeline and save results

In [49]:
def denoising(img, with_save):
    img = 1-img
    kernel = np.ones((7,7), np.uint8) 
    img_erosion = cv2.erode(img, kernel, iterations=1)
    if with_save:
        save(1-img_erosion, "img_erosion.jpg")
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    if with_save:
        save(1-img_dilation, "img_dilation.jpg")
    return 1-img_dilation

In [81]:
img_full = cv2.imread('r_1_11.jpg')
img_blurred = cv2.medianBlur(img_full, 7)

img_classified = detect(img_blurred, gnb_blurred).astype('float32')
save(img_classified, "img_classified.jpg")

#img_filtered = median_filter(img_classified)
#save(img_filtered, "img_filtered.jpg")
img_denoised = denoising(img_classified, True)

img_rotated = rotate(img_denoised, -2)
save(img_rotated, "img_rotated.jpg")

img_pallet, (x,y,mask_height) = find_pallet(img_rotated, 25, 100, 3)
save(img_pallet, "img_pallet.jpg")

img_full_marked = draw_pallet(img_full,x,y,mask_height)
save(img_full_marked, "img_full_marked.jpg", False)
center = calculate_center(x,y,mask_height)
print("center: ",center," x,y:",x,y," mask_height: ",mask_height)

saved img_classified.jpg
saved img_erosion.jpg
saved img_dilation.jpg
saved img_rotated.jpg
height:  25 percent:  0.2299442586399108
height:  28 percent:  0.2731914096090667
height:  31 percent:  0.3656442384405625
saved img_pallet.jpg
222 255
saved img_full_marked.jpg
center:  (308.0, 270.5)  x,y: 222 255  mask_height:  31


# Pipeline as function

In [77]:
def localise(path,filename):
    img_full = cv2.imread(path+filename)
    #img_blurred = cv2.medianBlur(img_full, 7)
    img_classified = detect(img_full, gnb).astype('float32')
    img_denoised = denoising(img_classified, False)
    #img_filtered = median_filter(img_classified)
    img_rotated = rotate(img_denoised, -2)
    img_pallet, (x,y,mask_height) = find_pallet(img_rotated, 25, 100, 3)
    img_full_marked = draw_pallet(img_full,x,y,mask_height)
    save(img_full_marked, path+"localised/"+filename, False)
    return [calculate_center(x,y,mask_height)]

In [78]:
def localize_all():
    centers = dict()
    path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/"
    filenames = [f for f in sorted(listdir(path)) if isfile(join(path, f))]
    for i in tqdm(range(len(filenames))):
        filename = filenames[i]
        print(path, filename)
        centers[filename]=localise(path,filename)
    np.save(path+"eval/pred_centers", centers)

In [79]:
localize_all()


  0%|          | 0/85 [00:00<?, ?it/s]

/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_0.jpg
height:  25 percent:  0.28964259951472227
height:  28 percent:  0.30975459175798736
height:  31 percent:  0.37293177461031923
height:  34 percent:  0.37401230106131433



  1%|          | 1/85 [00:16<23:02, 16.46s/it]

402 252
saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_1_0.jpg
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_1.jpg
height:  25 percent:  0.250172470325923
height:  28 percent:  0.3668765021057042
height:  31 percent:  0.42536080448683594



  2%|▏         | 2/85 [00:32<22:37, 16.35s/it]

...


 34%|███▍      | 29/85 [07:44<15:09, 16.24s/it]

126 246
saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_1_34.jpg
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_35.jpg
height:  25 percent:  0.13855072463768117
height:  31 percent:  0.17556323604710702
height:  34 percent:  0.19680851063829788



 35%|███▌      | 30/85 [08:01<14:51, 16.21s/it]

282 246
saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_1_35.jpg
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_36.jpg
height:  25 percent:  0.2702118171683389
height:  28 percent:  0.32302383524794387
height:  31 percent:  0.3909005525190821


KeyboardInterrupt: 